In [18]:
from datetime import datetime

import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import folium
import re
import random

In [3]:
df = pd.read_excel(io='data.xlsx')

In [33]:
df[['N','E']].iloc[random.randint(0, len(df))]

N    55°02'8.5922"N
E     50°52'13.68"Е
Name: 10235, dtype: object

Извлечение радиан

In [12]:
def to_rads(value) -> float:

    pattern = "(\d*)°(\d*)'(\d*\.*\d*)"
    val = re.findall(pattern=pattern, string=value)[0]
    return float(val[0]) + float(val[1])/60 + float(val[2])/3600

In [15]:
df['rad_N'] = df['N'].map(arg=to_rads, na_action='ignore')

In [34]:
df['rad_E'] = df['E'].map(arg=to_rads, na_action='ignore')

Извлечение периода

In [ ]:
geodata = df[['num','date', 'owner', 'INN', 'N', 'E']]
geodata['month'] = geodata['date'].dt.to_period('M')

In [ ]:
def coords(period):
    if period:
        start_date = period[0]
        end_date = period[1]
        mask = (geodata['month'] >= start_date) & (geodata['month'] <= end_date)
        return geodata[mask]
    else: return None

In [ ]:
delta = (pd.to_datetime('01/01/2020').to_period('M'), pd.to_datetime('01/01/2022').to_period('M'))
coords(delta)

In [ ]:
@interact(coords, dates=widgets.SelectionRangeSlider(
    index=
    options=
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}))

In [ ]:
len(geodata['month'][geodata['month'] > pd.to_datetime('2010-01-02').to_period('M')])

In [ ]:
def on_value_change(change):
    print(change)

selection_range_slider = widgets.SelectionRangeSlider(
    options=geodata['month'][geodata['month'] > pd.to_datetime('2020-01-02').to_period('M')],
    index = (200,1000),
    description='Dates',
    orientation='horizontal',
    layout={'width': '1000px'})

selection_range_slider

In [ ]:
widgets.interact(on_value_change, change=selection_range_slider)

In [ ]:
widgets.interact(coords, period=selection_range_slider)

In [66]:
location = df.dropna(subset=['rad_N'])

1. Сделай селектИнт и поквартально
2. Перевести координаты в градусы

In [79]:
def render():
    for n in location[['rad_N', 'rad_E']].itertuples(index=False):
        folium.Marker(location=n).add_to(m)

In [83]:
m = folium.Map(tiles="Stamen Toner", location=location[['rad_N', 'rad_E']].iloc[1])
render()


In [84]:
m